In [1]:
import julia
julia.install()

from pathlib import Path

# # Point to the top of the project relative to this script
def projectdir(*args):
    return str(Path.cwd().joinpath("..", "..", "..", *args).resolve())

In [2]:
from torch.utils.data import DataLoader
from julia import Main as jl
from sklearn.metrics import accuracy_score
import ipdb
import torch
# from torchvision.transforms import Lambda
from tqdm import tqdm

from torchvision.models import resnet50
from torchvision.models import ResNet50_Weights
from torchvision import models
from torchvision.models.feature_extraction import create_feature_extractor

class DDVFAStrategy():
    """DDVFA Strategy"""

    def __init__(self):
        jl.project_dir = projectdir()
        jl.eval("using Pkg; Pkg.activate(project_dir)")
        jl.eval("using AdaptiveResonance")
        jl.eval("art = DDVFA(rho_lb=0.4, rho_ub=0.75)")
        # jl.eval("art.config = DataConfig(0.0, 255.0, 28*28)")
        jl.eval("art.config = DataConfig(-1.0, 1.0, 28*28)")
        
        rn = resnet50()
        self.mod = create_feature_extractor(rn, {'layer4': 'layer4'})
        self.mod = self.mod.to('cuda')
        self.mod.eval()
        self.weights = ResNet50_Weights.DEFAULT
        self.preprocess = self.weights.transforms()
        # self.gs = Grayscale(num_output_channels=3)
        # self.trans = Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)
        # self.model = Resnet50Extractor()

    def ext_features(self, img):
        # prep = preprocess(img).unsqueeze(0)
        # print(img.shape)
        # rgbimg = self.gs(img)
        # rgbimg = Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
        # rgbimg = self.trans(img)
        # print(rgbimg.shape)
        # prep = self.preprocess(rgbimg)
        img = img.to('cuda')
        prep = self.preprocess(img)
        features = self.mod(prep)['layer4']

        avg_features = features.mean(dim=1).flatten().detach().cpu().numpy()
        # avg_features.flatten().detach().numpy()
        return avg_features

    def train(self, experience):
        train_dataset = experience.dataset
        t = experience.task_label
        train_data_loader = DataLoader(
            # train_dataset,
            dataset=train_dataset,
            pin_memory=True,
            # num_workers=1,
            # num_workers=4,
            # batch_size=128,
            batch_size=32,
            # transform=self.trans
        )
        print(experience.dataset.__len__())
        for mb in train_data_loader:
            # ld = mb[0].to('cuda')
            ld = mb[0]
            af = self.ext_features(ld)
            print(af.shape)
            # ipdb.set_trace()
            # pass
            # for sample, label, task in mb:
            # for sample in mb:
            #     self.ext_features(sample)

        # for sample in train_dataset:
        # n_samples = len(train_dataset)
        # for ix in range(n_samples):
        # for ix in tqdm(range(n_samples)):
            # ls, ll, lt = train_dataset[ix]
            # self.ext_features(ls)
            # self.model(ls)

            # # ipdb.set_trace()
            # jl.sample = ls.flatten().numpy()
            # jl.label = ll
            # jl.t = lt
            # jl.eval("train!(art, sample, y=label)")

    def eval(self, experience):
        eval_dataset = experience.dataset
        t = experience.task_label
        eval_data_loader = DataLoader(
            # eval_dataset, num_workers=4, batch_size=128
            eval_dataset, num_workers=4, batch_size=32
        )
        print(eval_data_loader)

        total = 0
        correct = 0
        for mb in eval_data_loader:
            for sample, label in mb:
        # for mb in eval_data_loader:
        # for sample in eval_dataset:
                jl.sample = sample
                jl.label = label
                y_hat = jl.eval("classify(art, sample)")
                total += 1
                if y_hat == label:
                    correct += 1
        
        return correct/total
        # j.samples = mb
        # y_hats = j.eval("classify(art, samples)")
        # accuracy_score(y_test, y_hat)

In [3]:
def print_allocated_memory():
   print("{:.2f} GB".format(torch.cuda.memory_allocated() / 1024 ** 3))

print_allocated_memory()
# torch.cuda.empty_cache()
# print_allocated_memory()

0.00 GB


In [4]:
from avalanche.benchmarks import SplitMNIST
from avalanche.benchmarks.classic import SplitCIFAR10
from torchvision.transforms import Lambda

trans = Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)
# Benchmark creation
benchmark = SplitMNIST(
    n_experiences=5,
    train_transform=trans,
    eval_transform=trans,
)
# benchmark = SplitCIFAR10(n_experiences=5, return_task_id=True)

# Create the Strategy Instance
cl_strategy = DDVFAStrategy()

# Training Loop
print('Starting experiment...')

for exp_id, experience in enumerate(benchmark.train_stream):
    print("Start of experience ", experience.current_experience)

    # print(experience.dataset)
    cl_strategy.train(experience)
    print('Training completed')

#     print('Computing accuracy on the current test set')
#     cl_strategy.eval(benchmark.test_stream[exp_id])

Starting experiment...
Start of experience  0
12080


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [82]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# img = read_image("test/assets/encode_jpeg/grace_hopper_517x606.jpg")
img = read_image("grace_hopper_517x606.jpg")

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\Sasha/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

bow tie: 14.4%


In [79]:
inp = torch.randn(2, 3, 224, 224)
model(inp)

{'4': tensor([[[[1.2357e+00, 0.0000e+00, 1.1311e+00,  ..., 0.0000e+00,
            1.3839e+00, 0.0000e+00],
           [2.0044e+00, 0.0000e+00, 0.0000e+00,  ..., 6.7144e-01,
            3.2935e+00, 0.0000e+00],
           [1.1263e-01, 1.4880e+00, 0.0000e+00,  ..., 2.7060e+00,
            2.0432e+00, 1.2406e+00],
           ...,
           [2.9631e+00, 2.8480e-01, 2.1829e+00,  ..., 0.0000e+00,
            3.0080e+00, 1.5533e+00],
           [1.4739e+00, 1.4666e+00, 5.5931e+00,  ..., 2.9052e+00,
            1.6959e+00, 0.0000e+00],
           [3.5369e-01, 2.3530e+00, 1.0774e+00,  ..., 8.4163e-01,
            1.2362e+00, 0.0000e+00]],
 
          [[5.0615e-02, 0.0000e+00, 0.0000e+00,  ..., 1.0539e+00,
            5.5839e-03, 1.0187e+00],
           [1.0786e+00, 5.3807e+00, 1.2345e+00,  ..., 1.2468e+00,
            9.6508e-01, 0.0000e+00],
           [1.2106e-01, 1.5680e+00, 0.0000e+00,  ..., 2.3732e+00,
            1.1339e+00, 2.0850e+00],
           ...,
           [1.6436e+00, 3.8410e-0

In [ ]:
"""
This example trains on Split CIFAR10 with Naive strategy.
In this example each experience has a different task label.
We use a multi-head model with a separate classifier for each task.
"""

import argparse
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.models import SimpleMLP, as_multitask
from avalanche.training.supervised import Naive


def main(args):

    # model
    model = SimpleMLP(input_size=32 * 32 * 3, num_classes=10)
    model = as_multitask(model, "classifier")

    # CL Benchmark Creation
    benchmark = SplitCIFAR10(n_experiences=5, return_task_id=True)
    train_stream = benchmark.train_stream
    test_stream = benchmark.test_stream

    # Prepare for training & testing
    optimizer = Adam(model.parameters(), lr=0.01)
    criterion = CrossEntropyLoss()

    # Choose a CL strategy
    strategy = Naive(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        train_mb_size=128,
        train_epochs=3,
        eval_mb_size=128,
        device=device,
    )

    # train and test loop
    for train_task in train_stream:
        strategy.train(train_task, num_workers=0)
        strategy.eval(test_stream)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--cuda",
        type=int,
        default=0,
        help="Select zero-indexed cuda device. -1 to use CPU.",
    )
    args = parser.parse_args()
    main(args)


In [ ]:
from avalanche.benchmarks import SplitMNIST
# from avalanche.benchmarks.utils.data_loader import GroupBalancedDataLoader
from avalanche.benchmarks.utils.data_loader import TaskBalancedDataLoader
benchmark = SplitMNIST(5, return_task_id=True)

dl = TaskBalancedDataLoader([exp.dataset for exp in benchmark.train_stream], batch_size=4)
for x, y, t in dl:
    print(t.tolist())
    break

AttributeError: 'list' object has no attribute 'targets_task_labels'